TODO:
- Reproduce the error for image loading
- Assert cam_R_w2c with reprojection
- How can I use @pytest.fixture and unit_test.mock (mock = MagicMock())

In [44]:
import os
import time
import json

from collections import OrderedDict
import yaml
import argparse

import torch
import numpy as np
import pandas as pd
import pickle as pkl
import logging
from cosypose.config import EXP_DIR, MEMORY, RESULTS_DIR, LOCAL_DATA_DIR

import cosypose.utils.tensor_collection as tc

from cosypose.datasets.datasets_cfg import make_scene_dataset, make_object_dataset


In [46]:
n_workers = 8
n_plotters = 8
n_views = 1

n_frames = None
scene_id = None
group_id = None
n_groups = None
skip_mv = n_views < 2
skip_predictions = False

object_set = 'bracket_assembly'
coarse_run_id = f'bracket_assembly_coarse-transnoise-zxyavg-616093'
refiner_run_id = 'bracket_assembly_refiner--558735'
n_coarse_iterations = 1
n_refiner_iterations = 2
config = ds_name = 'bracket_assembly'
comment = ''

In [47]:
scene_id = 0
n_rand = np.random.randint(1e10)
save_dir = RESULTS_DIR / f'{config}-n_views={n_views}-{comment}-{n_rand}'
scene_ds = make_scene_dataset(ds_name)
mask = scene_ds.frame_index['scene_id'] == scene_id
scene_ds.frame_index = scene_ds.frame_index[mask].reset_index(drop=True)

1 day, 22:57:48.369978 - Building index and loading annotations...


[Memory]169068.5s, 2817.8min: Loading build_index...


In [48]:
from cosypose.utils.logging import get_logger
logger = get_logger(__name__)

In [49]:
def load_custom_detection_from_gt():
    
    path_data_dir = LOCAL_DATA_DIR / 'bop_datasets' / 'bracket_assembly'
    path_scene_dir = os.path.join(path_data_dir, "train_pbr")
    scene_names = os.listdir(path_scene_dir)
    infos, poses, bboxes = [], [], []
    # debug_only = 0
    for scene_id, scene_name in enumerate(scene_names):
        path_scene_gt_info = os.path.join(path_scene_dir, scene_name, "scene_gt_info.json")
        path_scene_gt = os.path.join(path_scene_dir, scene_name, "scene_gt.json")
        path_scene_gt_camera = os.path.join(path_scene_dir, scene_name, "scene_camera.json")
        with open(path_scene_gt_info, "r") as f:
            json_data_gt_info = json.load(f)
        with open(path_scene_gt, "r") as f:
            json_data_gt = json.load(f)
        with open(path_scene_gt_camera, "r") as f:
            json_gt_camera = json.load(f)
        img_names_rgb = os.listdir(os.path.join(path_scene_dir, scene_name, "rgb"))
        # todo
        # scene_camera = load_scene_camera(path_scene_gt_camera)
        from cosypose.lib3d import Transform
        for img_id, img_name in enumerate(img_names_rgb[:-1]):
            if not f"{img_id}" in json_data_gt_info:
                continue
            if not f"{img_id}" in json_data_gt:
                continue
            if not f"{img_id}" in json_gt_camera:
                continue
            # cam_R_w2c = scene_camera[img_id]["cam_R_w2c"]
            # TODO
            cam_R_w2c = json_gt_camera[f"{img_id}"]["cam_R_w2c"]
            cam_t_w2c = json_gt_camera[f"{img_id}"]["cam_t_w2c"]
            row0 = [cam_R_w2c[0], cam_R_w2c[1], cam_R_w2c[2], cam_t_w2c[0]] 
            row1 = [cam_R_w2c[3], cam_R_w2c[4], cam_R_w2c[5], cam_t_w2c[1]]
            row2 = [cam_R_w2c[6], cam_R_w2c[7], cam_R_w2c[8], cam_t_w2c[2]]           
            row3 = [0, 0, 0, 1]
            cam_rot_loc_mat = np.asarray([row0, row1, row2, row3])


            if 'cam_R_w2c' in json_gt_camera[f"{img_id}"]:
                RC0 = np.array(json_gt_camera[f"{img_id}"]['cam_R_w2c']).reshape(3, 3)
                tC0 = np.array(json_gt_camera[f"{img_id}"]['cam_t_w2c'])# * 0.001
                TC0 = Transform(RC0, tC0)
                T0C = TC0.inverse()
                T0C = T0C.toHomogeneousMatrix()
            for label_idx, label in enumerate(json_data_gt[f"{img_id}"]):
                obj_id = label["obj_id"] # int
                list_bbox = json_data_gt_info[f"{img_id}"][label_idx]["bbox_obj"] # TODO: ?
                xmin = list_bbox[0]
                ymin = list_bbox[1]
                xmax = list_bbox[2]
                ymax = list_bbox[1] + list_bbox[3]
                list_bbox = [xmin, ymin, xmax, ymax]
                list_rot  = json_data_gt[f"{img_id}"][label_idx]["cam_R_m2c"]
                list_loc  = json_data_gt[f"{img_id}"][label_idx]["cam_t_m2c"]
                # RCO = np.array(json_data_gt[f"{img_id}"][label_idx]['cam_R_m2c']).reshape(3, 3)
                # tCO = np.array(json_data_gt[f"{img_id}"][label_idx]['cam_t_m2c']) #* 0.001
                # TCO = Transform(RCO, tCO)
                # T0O = T0C * TCO
                # T0O = T0O.toHomogeneousMatrix()

                row0 = [list_rot[0], list_rot[1], list_rot[2], list_loc[0]] 
                row1 = [list_rot[3], list_rot[4], list_rot[5], list_loc[1]]
                row2 = [list_rot[6], list_rot[7], list_rot[8], list_loc[2]]
                row3 = [0, 0, 0, 1]
                rot_loc_mat = np.asarray([row0, row1, row2, row3])
                rot_loc_mat = np.matmul(np.linalg.inv(cam_rot_loc_mat), rot_loc_mat)
                # if scene_id == 0 and img_id == 20:
                #     print("label_idx",label_idx)
                #     print("rot_loc_mat", rot_loc_mat)
                infos.append(dict(
                        scene_id=scene_id,
                        view_id=img_id,
                        score=1,
                        label=f"obj_{obj_id:06d}",
                    ))
                # poses.append(T0O)
                poses.append(rot_loc_mat)
                bboxes.append(list_bbox)
    data = tc.PandasTensorCollection(
        infos=pd.DataFrame(infos),
        poses=torch.as_tensor(np.stack(poses)).float(),
        bboxes=torch.as_tensor(np.stack(bboxes)).float(),
    ).cpu()
    return data
bracket_detections = load_custom_detection_from_gt(ds_name).cpu()
# from IPython.display import clear_output
# clear_output()

In [50]:
from pathlib import Path
import torch
LOCAL_DATA = Path('/home/ubuntu/synthetic_pose_estimation/cosypose/local_data')
DEBUG_DATA_DIR = LOCAL_DATA / 'debug_data'
debug_file = open(DEBUG_DATA_DIR/'debug_iter=1.pth.tar','rb')
debug_data = torch.load(debug_file)

In [51]:
debug_data['images']

tensor([[[[0.1176, 0.1333, 0.1373,  ..., 0.0824, 0.0706, 0.0824],
          [0.1412, 0.1451, 0.1451,  ..., 0.0824, 0.0980, 0.1216],
          [0.1529, 0.1490, 0.1412,  ..., 0.0863, 0.0941, 0.1176],
          ...,
          [0.1020, 0.1255, 0.0000,  ..., 0.0039, 0.0000, 0.0000],
          [0.0980, 0.1255, 0.0000,  ..., 0.0039, 0.0000, 0.0000],
          [0.0824, 0.1137, 0.0000,  ..., 0.0039, 0.0000, 0.0000]],

         [[0.4039, 0.4196, 0.4314,  ..., 0.5647, 0.5961, 0.5529],
          [0.4353, 0.4392, 0.4392,  ..., 0.5647, 0.6118, 0.5922],
          [0.4471, 0.4431, 0.4353,  ..., 0.5608, 0.6078, 0.5882],
          ...,
          [0.4157, 0.3529, 0.0941,  ..., 0.0000, 0.0000, 0.0039],
          [0.3686, 0.3098, 0.0667,  ..., 0.0000, 0.0000, 0.0039],
          [0.3176, 0.2627, 0.0314,  ..., 0.0000, 0.0000, 0.0039]],

         [[0.4314, 0.4471, 0.4549,  ..., 0.6863, 0.7098, 0.6510],
          [0.4588, 0.4627, 0.4627,  ..., 0.6863, 0.7294, 0.6902],
          [0.4706, 0.4667, 0.4588,  ..., 0

In [52]:
len(bracket_detections)

14917

In [53]:
def load_models(coarse_run_id, refiner_run_id=None, n_workers=8, object_set='tless'):
    if object_set == 'bracket_assembly':
        object_ds_name, urdf_ds_name = 'bracket_assembly', 'bracket_assembly'
    
    object_ds = make_object_dataset(object_ds_name)
    mesh_db = MeshDataBase.from_object_ds(object_ds)
    renderer = BulletBatchRenderer(object_set=urdf_ds_name, n_workers=n_workers)
    mesh_db_batched = mesh_db.batched().cuda()

    def load_model(run_id):
        if run_id is None:
            return
        run_dir = EXP_DIR / run_id
        cfg = yaml.unsafe_load((run_dir / 'config.yaml').read_text())
        cfg = check_update_config(cfg)
        if cfg.train_refiner:
            model = create_model_refiner(cfg, renderer=renderer, mesh_db=mesh_db_batched)
            ckpt = torch.load(run_dir / 'checkpoint.pth.tar')
        else:
            model = create_model_coarse(cfg, renderer=renderer, mesh_db=mesh_db_batched)
            ckpt = torch.load(run_dir / 'checkpoint.pth.tar')
        ckpt = ckpt['state_dict']
        model.load_state_dict(ckpt)
        model = model.cuda().eval()
        model.cfg = cfg
        return model

    coarse_model = load_model(coarse_run_id)
    refiner_model = load_model(refiner_run_id)
    model = CoarseRefinePosePredictor(coarse_model=coarse_model,
                                      refiner_model=refiner_model,
                                      bsz_objects=1)
    return model, mesh_db

In [54]:
n_frames = 20 # None
if scene_id is not None:
    mask = scene_ds.frame_index['scene_id'] == scene_id
    scene_ds.frame_index = scene_ds.frame_index[mask].reset_index(drop=True)
if n_frames is not None:
    # scene_ds.frame_index = scene_ds.frame_index.reset_index(drop=True)[:n_frames]
    scene_ds.frame_index = scene_ds.frame_index.reset_index(drop=True)[n_frames:n_frames+1]

In [55]:
len(scene_ds)

1

In [56]:
def get_pose_meters(scene_ds):
    ds_name = scene_ds.name

    compute_add = False
    spheres_overlap_check = True
    large_match_threshold_diameter_ratio = 0.5
    print("ds_name", ds_name)
    if ds_name == 'tless.primesense.test.bop19':
        targets_filename = 'test_targets_bop19.json'
        visib_gt_min = -1
        n_top = -1  # Given by targets
    elif ds_name == 'tless.primesense.test':
        targets_filename = 'all_target_tless.json'
        n_top = 1
        visib_gt_min = 0.1
    elif 'ycbv' in ds_name:
        compute_add = True
        visib_gt_min = -1
        targets_filename = None
        n_top = 1
        spheres_overlap_check = False
    elif 'bracket_assembly' in ds_name:
        targets_filename = None
        visib_gt_min = -1
        n_top = 1  # Given by targets
        spheres_overlap_check = False
    else:
        raise ValueError

    if 'tless' in ds_name:
        object_ds_name = 'tless.eval'
    elif 'ycbv' in ds_name:
        object_ds_name = 'ycbv.bop-compat.eval'  # This is important for definition of symmetric objects
    elif 'bracket_assembly' in ds_name:
        object_ds_name = 'bracket_assembly'
    else:
        raise ValueError

    if targets_filename is not None:
        targets_path = scene_ds.ds_dir / targets_filename
        targets = pd.read_json(targets_path)
        targets = remap_bop_targets(targets)
    else:
        targets = None

    object_ds = make_object_dataset(object_ds_name)
    print("object_ds_name", object_ds_name)
    mesh_db = MeshDataBase.from_object_ds(object_ds)

    error_types = ['ADD-S'] + (['ADD(-S)'] if compute_add else [])

    base_kwargs = dict(
        mesh_db=mesh_db,
        # exact_meshes=True,
        # sample_n_points=None,
        exact_meshes=False,
        sample_n_points=100,
        errors_bsz=1,

        # BOP-Like parameters
        n_top=n_top,
        visib_gt_min=visib_gt_min,
        targets=targets,
        spheres_overlap_check=spheres_overlap_check,
    )

    meters = dict()
    for error_type in error_types:
        # For measuring ADD-S AUC on T-LESS and average errors on ycbv/tless.
        meters[f'{error_type}_ntop=BOP_matching=OVERLAP'] = PoseErrorMeter(
            error_type=error_type, consider_all_predictions=False,
            match_threshold=large_match_threshold_diameter_ratio,
            report_error_stats=True, report_error_AUC=True, **base_kwargs)

        if 'ycbv' in ds_name:
            # For fair comparison with PoseCNN/DeepIM on YCB-Video ADD(-S) AUC
            meters[f'{error_type}_ntop=1_matching=CLASS'] = PoseErrorMeter(
                error_type=error_type, consider_all_predictions=False,
                match_threshold=np.inf,
                report_error_stats=False, report_error_AUC=True, **base_kwargs)

        if 'tless' in ds_name:
            meters.update({f'{error_type}_ntop=BOP_matching=BOP':  # For ADD-S<0.1d
                           PoseErrorMeter(error_type=error_type, match_threshold=0.1, **base_kwargs),

                           f'{error_type}_ntop=ALL_matching=BOP':  # For mAP
                           PoseErrorMeter(error_type=error_type, match_threshold=0.1,
                                          consider_all_predictions=True,
                                          report_AP=True, **base_kwargs)})
    return meters

In [57]:
# Predictions
predictor, mesh_db = load_models(coarse_run_id, refiner_run_id, n_workers=n_plotters, object_set=object_set)

mv_predictor = MultiviewScenePredictor(mesh_db)
base_pred_kwargs = dict(
    n_coarse_iterations=n_coarse_iterations,
    n_refiner_iterations=n_refiner_iterations,
    skip_mv=skip_mv,
    pose_predictor=predictor,
    mv_predictor=mv_predictor,
)

if 'bracket_assembly' in ds_name:
    bracket_detections = load_custom_detection_from_gt().cpu()
    pred_kwargs = {
        'pix2pose_detections': dict(
            detections=bracket_detections,
            **base_pred_kwargs
        )
    }

scene_ds_pred = MultiViewWrapper(scene_ds, n_views=n_views)
if group_id is not None:
    mask = scene_ds_pred.frame_index['group_id'] == group_id
    scene_ds_pred.frame_index = scene_ds_pred.frame_index[mask].reset_index(drop=True)
elif n_groups is not None:
    scene_ds_pred.frame_index = scene_ds_pred.frame_index[:n_groups]

pred_runner = MultiviewPredictionRunner(
    scene_ds_pred, batch_size=1, n_workers=n_workers,
    cache_data=len(pred_kwargs) > 1)

all_predictions = dict()
for pred_prefix, pred_kwargs_n in pred_kwargs.items():
    preds = pred_runner.get_predictions(**pred_kwargs_n)
    # logger.info(f"preds,{pred_prefix}, {preds}")
    for preds_name, preds_n in preds.items():
        all_predictions[f'{pred_prefix}/{preds_name}'] = preds_n

logger.info("Done with predictions")
# torch.distributed.barrier()

# Evaluation
predictions_to_evaluate = set()
if 'ycbv' in ds_name:
    det_key = 'posecnn_init'
    all_predictions['posecnn'] = posecnn_detections
    predictions_to_evaluate.add('posecnn')
    predictions_to_evaluate.add(f'{det_key}/refiner/iteration={n_refiner_iterations}')
elif 'tless' in ds_name:
    det_key = 'pix2pose_detections'
    predictions_to_evaluate.add(f'{det_key}/refiner/iteration={n_refiner_iterations}')
elif 'bracket_assembly' in ds_name: # BOP dataset
    det_key = 'pix2pose_detections'
    predictions_to_evaluate.add(f'{det_key}/coarse/iteration=1')
    predictions_to_evaluate.add(f'{det_key}/refiner/iteration={n_refiner_iterations}')
else:
    raise ValueError(ds_name)

if n_views > 1:
    for k in [
            # f'ba_input',
            # f'ba_output',
            f'ba_output+all_cand'
    ]:
        predictions_to_evaluate.add(f'{det_key}/{k}')

all_predictions = OrderedDict({k: v for k, v in sorted(all_predictions.items(), key=lambda item: item[0])})
# Evaluation.
meters = get_pose_meters(scene_ds)
mv_group_ids = list(iter(pred_runner.sampler))
scene_ds_ids = np.concatenate(scene_ds_pred.frame_index.loc[mv_group_ids, 'scene_ds_ids'].values)
sampler = ListSampler(scene_ds_ids)
eval_runner = PoseEvaluation(scene_ds, meters, n_workers=n_workers,
                                cache_data=True, batch_size=1, sampler=sampler)

1 day, 22:57:55.099552 - Backbone: efficientnet-b3
1 day, 22:57:55.534823 - Backbone: efficientnet-b3
EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)
EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)


Loaded EGL 1.5 after reload.
Loaded EGL 1.5 after reload.


EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)
EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)
EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)


Loaded EGL 1.5 after reload.
Loaded EGL 1.5 after reload.
Loaded EGL 1.5 after reload.
Loaded EGL 1.5 after reload.


EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)
EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)
EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)


...

100%|██████████| 1/1 [00:06<00:00,  6.42s/it]

torch.Size([1, 3, 540, 720]) torch.Size([1, 3, 240, 320])
ven = NVIDIA Corporation
torch.Size([1, 3, 540, 720]) torch.Size([1, 3, 240, 320])



1 day, 22:58:02.165951 - preds,pix2pose_detections, {'detections': PandasTensorCollection(
    poses: torch.Size([7, 4, 4]) torch.float32 cpu,
    bboxes: torch.Size([7, 4]) torch.float32 cpu,
----------------------------------------
    infos:
   scene_id  view_id  score       label  det_id  batch_im_id  group_id
0         0       20      1  obj_000000     140            0         0
1         0       20      1  obj_000004     141            0         0
2         0       20      1  obj_000005     142            0         0
3         0       20      1  obj_000006     143            0         0
4         0       20      1  obj_000007     144            0         0
5         0       20      1  obj_000008     145            0         0
6         0       20      1  obj_000009     146            0         0
), 'coarse/iteration=1': PandasTensorCollection(
    poses: torch.Size([7, 4, 4]) torch.float32 cpu,
    poses_input: torch.Size([7, 4, 4]) torch.float32 cpu,
    K_crop: torch.Size([7, 

detections PandasTensorCollection(
    poses: torch.Size([14917, 4, 4]) torch.float32 cpu,
    bboxes: torch.Size([14917, 4]) torch.float32 cpu,
----------------------------------------
    infos:
       scene_id  view_id  score       label
0             0        0      1  obj_000000
1             0        0      1  obj_000004
2             0        0      1  obj_000005
3             0        0      1  obj_000006
4             0        0      1  obj_000007
...         ...      ...    ...         ...
14912         2      998      1  obj_000005
14913         2      998      1  obj_000006
14914         2      998      1  obj_000007
14915         2      998      1  obj_000008
14916         2      998      1  obj_000009

[14917 rows x 4 columns]
)
ds_name bracket_assembly
object_ds_name bracket_assembly


100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


In [58]:
scene_id = 0
frame_id = 20
object_ds_name = 'bracket_assembly'
object_ds = make_object_dataset(object_ds_name)
mesh_db = MeshDataBase.from_object_ds(object_ds).batched().cuda().float()
meshes = mesh_db.select(labels)
points = meshes.sample_points(2000, deterministic=True)
uv = project_points(points, K, TCO)
boxes_rend = boxes_from_uv(uv)
boxes_crop, images_cropped = deepim_crops(
    images=images, obs_boxes=boxes_rend, K=K,
    TCO_pred=TCO, O_vertices=points, output_size=self.render_size, lamb=1.4
)
plot(image)
plot(boxes)

NameError: name 'labels' is not defined